In [13]:
import requests
import re
import pandas as pd

#输入剧集地址
drama_id ='   https://www.missevan.com/mdrama/76879   '

drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id=  '
url = drama_url +drama_id.split("/")[4]
drama_result =requests.get(url).text
pattern1 =re.compile('sound_id":(.+?),"type')
pattern2 = re.compile('"need_pay":(.+?),"')
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)
pay_type =pay_type[1:]
drama = pd.DataFrame({'sound_id': sound_id,'pay_type':pay_type})
sound_id =list(drama['sound_id'][drama['pay_type']=='1'])
sound_url = 'http://www.missevan.com/sound/getdm?soundid='

for i in range(0,len(sound_id)):
  url=sound_url+sound_id[i]
  sound_result=requests.get(url).text
  pattern3=re.compile(r'p="(.+?)"')
  dm =re.findall(pattern3, sound_result)
  if i==0:
    dm1=pd.DataFrame({'dm':dm})
    dms=dm1['dm'].str.split('>',expand=True)[0]
    uid =dms.str.split(',',expand=True)[6]
    dm1['uid']=uid
  else:
    dm2=pd.DataFrame({'dm':dm})
    dms_i=dm2['dm'].str.split('>',expand=True)[0]
    uid =dms.str.split(',',expand=True)[6]
    dm2['uid']=uid
    dm1=dm1.append(dm2)

print("本剧实时付费ID为：", len(dm1['uid'].unique()))
print("本剧实时付费弹幕数量为：",len(dm1['uid']))

AttributeError: 'DataFrame' object has no attribute 'append'

猫耳付费id抓取，正确版


In [16]:
import requests
import re
import pandas as pd

# 输入剧集地址
drama_id = 'https://www.missevan.com/mdrama/75026'

# 构建 URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
url = drama_url + drama_id.split("/")[4]

# 请求剧集数据
drama_result = requests.get(url).text

# 使用正则表达式提取 sound_id 和 pay_type
pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)

# 去掉第一个非付费项
pay_type = pay_type[1:]

# 创建 DataFrame
drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})

# 筛选付费 sound_id
sound_id = list(drama['sound_id'][drama['pay_type'] == '1'])

sound_url = 'http://www.missevan.com/sound/getdm?soundid='
dm1 = pd.DataFrame()

# 循环处理每个 sound_id，获取弹幕信息
for i in range(len(sound_id)):
    url = sound_url + sound_id[i]
    sound_result = requests.get(url).text

    # 提取弹幕内容
    pattern3 = re.compile(r'p="(.+?)"')
    dm = re.findall(pattern3, sound_result)

    # 将弹幕数据存储到 DataFrame
    dm_temp = pd.DataFrame({'dm': dm})
    dms_split = dm_temp['dm'].str.split('>', expand=True)[0]
    uid = dms_split.str.split(',', expand=True)[6]
    dm_temp['uid'] = uid

    # 合并数据，使用 pd.concat() 替代 append()
    if i == 0:
        dm1 = dm_temp
    else:
        dm1 = pd.concat([dm1, dm_temp], ignore_index=True)

# 输出不重复的 UID 数量和总弹幕数量
print("本剧实时付费ID为：", len(dm1['uid'].unique()))
print("本剧实时付费弹幕数量为：", len(dm1['uid']))


本剧实时付费ID为： 16906
本剧实时付费弹幕数量为： 447497


漫播弹幕id抓取


In [20]:
import requests
import re
import json

# 输入漫播剧集的任意一集链接（PC 或 APP 端）
url = "https://manbo.kilakila.cn/manbo/pc/detail?id=1944764730950287431&collectId=1962350882565652519"

# 判断链接类型
if 'pc' in url:
    detail_id = url.split('Id=')[1].split('&')[0]  # 提取 detail_id
    new_url = "https://manbo.hongdoulive.com/web_manbo/dramaSetDetail?dramaSetId=" + detail_id
    temp = 'data'
else:
    episode_id = url.split('id=')[1][:19]  # 提取 episode_id
    radio_id = url.split('DramaId=')[1][:19]  # 提取 radio_id
    new_url = "https://manbo.hongrenshuo.com.cn/api/v207/radio/drama/set/h5/detail?radioDramaSetId=%s&radioDramaId=%s" % (episode_id, radio_id)
    temp = 'b'


r= requests.get(new_url).text
r= str(json.loads(r)[temp]['radioDramaResp']['setRespList'])
# 请求新的 URL，获取剧集信息
#response = requests.get(new_url).text

# 解析 JSON 数据，提取付费集信息
#r = json.loads(response)
pattern1 = re.compile("'vipFree': (.+?),")
vipFree = re.findall(pattern1, r)
num = len(vipFree)

pattern2 = re.compile("'setId': (.+?),")
setId = re.findall(pattern2, r)

# 找到所有免费集或付费集的 ID
episodes = [setId[i] for i in range(num) if vipFree[i] == '1']

# 如果没有免费集，查找付费集
if len(episodes) == 0:
    pattern3 = re.compile("'payType': (.+?),")
    payType = re.findall(pattern3, r)
    episodes = [setId[i] for i in range(num) if payType[i] == '1']

# 弹幕列表
danmu = []

# 获取每集的弹幕信息
for each in episodes:
    next_url = "https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=%s&startTime=0&endTime=10000000" % each
    print(next_url)
    danmakuList = json.loads(requests.get(next_url).content)['b']['danmakuList']

    # 提取每条弹幕的 eid
    for j in danmakuList:
        danmu.append(j['eid'])

# 输出不重复的弹幕 ID 数量和总弹幕数量
print("付费集ID数：%d" % len(set(danmu)))
print("付费集弹幕数：%d" % len(danmu))


https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1972503008432357537&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1975132550619725955&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1977715801649578080&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1980245223854309438&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1982901424903684266&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1985499098433716288&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1988100293836931134&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/a